In [2]:
import os
from utils import save_reorganize_response_to_csv

result_file = "/home/lym/PbCirculation/MarinePbFusion/outputs/pdf_outputs/qwen_output_2.1/4_reorganize_tables_data/results_cleaned_1.csv"
# result_file = "/home/lym/PbCirculation/MarinePbFusion/agent/outputs/pdf_outputs/qwen_output_2.0/4_reorganize_tables_data/results_cleaned_2.csv"
output_dir = "/home/lym/PbCirculation/MarinePbFusion/outputs/pdf_outputs/qwen_output_2.1/4_reorganize_tables_data"
tables_output_dir = os.path.join(output_dir, "reorganize_tables_csv")
os.makedirs(tables_output_dir, exist_ok=True)
total, success, success_none, failed_incomplete, failed_format = save_reorganize_response_to_csv(result_file, tables_output_dir)

print(f"Total: {total}")
print(f"Success: {success}")
print(f"Success (None): {success_none}")
print(f"Failed (Incomplete): {failed_incomplete}")
print(f"Failed (Format): {failed_format}")

Total: 2042
Success: 160
Success (None): 1820
Failed (Incomplete): 31
Failed (Format): 31


In [8]:
import pandas as pd
import re
from typing import Dict, Tuple
import yaml
from pathlib import Path

def is_pb_related(column_name: str) -> bool:
    """Check if the column name is related to Pb"""
    # return re.search(r'\b(?:pb|lead)\b', column_name, re.IGNORECASE) is not None
    return 'pb' in column_name.lower() or 'lead' in column_name.lower()

def match_pb_column(column_name: str) -> str:
    """Rule-based matching logic"""
    numbers = re.findall(r'\d+', column_name)
    unit = extract_unit(column_name)
    
    # Isotope ratio detection
    ratios = [('206', '204'), ('206', '207'), ('208', '206'),
                ('207', '204'), ('208', '207'), ('208', '204')]
    for num, den in ratios:
        if num in numbers and den in numbers:
            return f'Pb_{num}_{den}'
    
    if '210' in numbers and unit:
        return 'Pb_210_CONC [mBq/kg]'
    return 'Pb_CONC [pmol/kg]'

def extract_unit(column_name: str) -> str:
    """Extract unit from column name in the form of [unit] or (unit)"""
    match = re.search(r'\[([^\]]+)\]|\(([^\)]+)\)', column_name)
    return match.group(1) if match else None

def rule_based_mapping(csv_path, params):
    """Rule-based mapping implementation"""
    df = pd.read_csv(csv_path)
    mapped_df = pd.DataFrame(index=df.index, columns=params['standard_columns'])
    mapping = {}
    
    # Process mandatory columns
    for col in params['mandatory_columns']:
        if col in df.columns:
            mapped_df[col] = df[col]
            mapping[col] = col
            print(f"Direct mapped mandatory column: {col}")
        else:
            print(f"Missing mandatory column: {col}")

    # Process Pb columns
    pb_cols = [c for c in df.columns if is_pb_related(c)]
    for orig_col in pb_cols:
        std_col = match_pb_column(orig_col)
        if std_col:
            if std_col in mapping:
                print(f"Duplicate mapping for {std_col}, overwriting")
            mapped_df[std_col] = df[orig_col]
            mapping[std_col] = orig_col
            print(f"Mapped {orig_col} → {std_col}")
            
    return mapped_df, mapping

# Load parameters
file_path = "/home/lym/PbCirculation/MarinePbFusion/agent/table_Processor/config/parameters.yaml"
with open(file_path, "r", encoding="utf-8") as f:
    params = yaml.safe_load(f)

input_dir = "/home/lym/PbCirculation/MarinePbFusion/agent/outputs/pdf_outputs/qwen_output_2.0/4_reorganize_tables_data/reorganize_tables_csv"

# Initialize counters for Pb-related columns
print(params['standard_columns'])
column_counts = {col: 0 for col in params['standard_columns'] if is_pb_related(col)}
print(column_counts)
total_data_points = 0
processed_files = 0

# Process all CSV files
for csv_file in Path(input_dir).glob('*.csv'):
    try:
        df = pd.read_csv(csv_file)
        processed_files += 1
        
        # Apply rule-based mapping to get standardized columns
        mapped_df = pd.DataFrame(index=df.index)
        mapping = {}
        
        # Find Pb-related columns in the original data
        pb_cols = [c for c in df.columns if is_pb_related(c)]
        for orig_col in pb_cols:
            std_col = match_pb_column(orig_col)
            if std_col in params['standard_columns']:
                mapped_df[std_col] = pd.to_numeric(df[orig_col], errors='coerce')
                mapping[std_col] = orig_col
                
        # Count non-NA values in each Pb-related column
        for col in column_counts.keys():
            if col in mapped_df.columns:
                non_na_count = mapped_df[col].count()  # Count non-NA values
                column_counts[col] += non_na_count
                total_data_points += non_na_count
        
    except Exception as e:
        print(f"Error processing {csv_file}: {str(e)}")

# Print results
print("\nCount of Pb-related data points by column:")
print("-" * 50)
for col, count in column_counts.items():
    print(f"{col}: {count:,} data points")

print("\nSummary:")
print("-" * 50)
print(f"Total Pb-related data points: {total_data_points:,}")
print(f"Total CSV files processed: {processed_files}")


['Longitude [degrees_east]', 'Latitude [degrees_north]', 'DEPTH [m]', 'Pb_CONC [pmol/kg]', 'Pb_210_CONC [mBq/kg]', 'Pb_206_204', 'Pb_206_207', 'Pb_208_206', 'Pb_207_204', 'Pb_208_207', 'Pb_208_204']
{'Pb_CONC [pmol/kg]': 0, 'Pb_210_CONC [mBq/kg]': 0, 'Pb_206_204': 0, 'Pb_206_207': 0, 'Pb_208_206': 0, 'Pb_207_204': 0, 'Pb_208_207': 0, 'Pb_208_204': 0}

Count of Pb-related data points by column:
--------------------------------------------------
Pb_CONC [pmol/kg]: 2,296 data points
Pb_210_CONC [mBq/kg]: 386 data points
Pb_206_204: 495 data points
Pb_206_207: 279 data points
Pb_208_206: 87 data points
Pb_207_204: 465 data points
Pb_208_207: 110 data points
Pb_208_204: 465 data points

Summary:
--------------------------------------------------
Total Pb-related data points: 4,583
Total CSV files processed: 221


In [3]:
import csv
from utils import save_llm_response_to_csv
import logging

result_path = './outputs/qwen_output/4_reconstruct_tables_data/results.csv'
output_dir = './outputs/qwen_output/4_reconstruct_tables_data/reconstruct_csv_tables'

# Initialize counters
total_rows = 0
success_count = 0
failed_count = 0

# Read CSV file and process each row
with open(result_path, 'r', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    
    for row in reader:
        total_rows += 1
        
        # Extract text and pdf_path from current row
        text = row.get('reconstruct_tables_data_response', '')
        pdf_path = row.get('pdf_path', '')
        
        if not text or not pdf_path:
            logging.warning(f"Missing data in row {total_rows}: text={bool(text)}, pdf_path={bool(pdf_path)}")
            failed_count += 1
            continue
        
        # Process the row and save CSV
        output_path = save_llm_response_to_csv(text, pdf_path, output_dir)
        
        if output_path:
            success_count += 1
            logging.info(f"Successfully processed {pdf_path} -> {output_path}")
        else:
            failed_count += 1
            logging.error(f"Failed to process {pdf_path}")

# Print summary
logging.info(f"Processing complete: {success_count} successful, {failed_count} failed out of {total_rows} total")
print(f"Processing complete: {success_count} successful, {failed_count} failed out of {total_rows} total")

ERROR:root:Failed to process /nasdata/datasets/scihub/75300000/10.21577/0103-5053.20180228.pdf
ERROR:root:Failed to process /nasdata/datasets/scihub/16500000/10.1016/j.envpol.2005.10.042.pdf
ERROR:root:Failed to process /nasdata/datasets/scihub/17300000/10.1016/s0016-7037%2801%2900779-7.pdf
ERROR:root:Failed to process /nasdata/datasets/scihub/73900000/10.1016/j.marpolbul.2019.01.026.pdf
ERROR:root:Failed to process /nasdata/datasets/scihub/44900000/10.2113/gsecongeo.95.7.1473.pdf
ERROR:root:Failed to process /nasdata/datasets/scihub/73300000/10.14233/ajchem.2013.13598.pdf
ERROR:root:Failed to process /nasdata/datasets/scihub/47300000/10.1016/j.csr.2015.09.004.pdf
ERROR:root:Failed to process /nasdata/datasets/scihub/02800000/10.1016/0304-4203%2894%2990080-9.pdf
ERROR:root:Failed to process /nasdata/datasets/scihub/71200000/10.1016/b978-0-08-022960-7.50007-5.pdf
ERROR:root:Failed to process /nasdata/datasets/scihub/23300000/10.1016/j.marpolbul.2013.06.044.pdf
ERROR:root:Failed to proce

Processing complete: 1411 successful, 231 failed out of 1642 total


In [4]:
import pandas as pd
import os
import glob
import numpy as np

def validate_and_clean_csv(csv_path, output_dir):
    """Validates and cleans CSV files according to specified standards."""
    try:
        # Read the CSV file
        df = pd.read_csv(csv_path)
        
        # Check if required columns exist
        required_columns = ["Longitude [degrees_east]", "Latitude [degrees_north]", "DEPTH [m]"]
        if not all(col in df.columns for col in required_columns):
            print(f"Missing required columns in {csv_path}. Skipping file.")
            return 0
        
        # Find Pb-related columns
        pb_columns = [col for col in df.columns if "Pb" in col]
        if not pb_columns:
            print(f"No Pb-related columns found in {csv_path}. Skipping file.")
            return 0
        
        # Keep only the required columns and Pb-related columns
        columns_to_keep = required_columns + pb_columns
        df = df[columns_to_keep]
        
        # Convert columns to numeric, errors become NaN
        for col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        
        # Apply data validation rules
        valid_longitude = (df["Longitude [degrees_east]"] >= -180) & (df["Longitude [degrees_east]"] <= 180)
        valid_latitude = (df["Latitude [degrees_north]"] >= -90) & (df["Latitude [degrees_north]"] <= 90)
        valid_depth = df["DEPTH [m]"] > 0

        # ========================================
        # Additional validation rule: both longitude and latitude cannot be 0
        not_zero_coords = ~((df["Longitude [degrees_east]"] == 0) & (df["Latitude [degrees_north]"] == 0))
        # Filter out specific invalid combinations
        invalid_combinations = (
            # Specific invalid combination 1
            ((df["Longitude [degrees_east]"] == 30) & 
             (df["Latitude [degrees_north]"] == 45) & 
             (df["DEPTH [m]"] == 100)) |
            
            # Specific invalid combination 2
            ((df["Longitude [degrees_east]"] == -110) & 
             (df["Latitude [degrees_north]"] == -30) & 
             (df["DEPTH [m]"] == 200)) |
            
            # Specific invalid combination 3
            ((df["Longitude [degrees_east]"] == 50) & 
             (df["Latitude [degrees_north]"] == 20) & 
             (df["DEPTH [m]"] == 150))
        )
        valid_combinations = ~invalid_combinations
        # ========================================
        
        # Check for NaN values in all columns
        no_nans = ~df.isna().any(axis=1)
        
        # Combine all validation criteria
        valid_rows = (valid_longitude & 
                      valid_latitude & 
                      valid_depth & 
                      not_zero_coords & 
                      valid_combinations & 
                      no_nans)
        
        # Filter the dataframe to keep only valid rows
        cleaned_df = df[valid_rows]
        
        # Check if any valid rows remain
        if cleaned_df.empty:
            print(f"No valid data remains in {csv_path} after cleaning. Skipping file.")
            return 0
        
        # Save the cleaned data
        os.makedirs(output_dir, exist_ok=True)
        output_path = os.path.join(output_dir, os.path.basename(csv_path))
        cleaned_df.to_csv(output_path, index=False)
        
        # Calculate the number of valid Pb data points
        # Each row with N Pb columns counts as N data points
        valid_pb_count = len(cleaned_df) * len(pb_columns)
        
        return valid_pb_count
    
    except Exception as e:
        print(f"Error processing {csv_path}: {str(e)}")
        return 0


input_dir = "/home/lym/PbCirculation/MarinePbFusion/agent/pdf_processor/outputs/qwen_output/4_reconstruct_tables_data/reconstruct_csv_tables"
output_dir = "/home/lym/PbCirculation/MarinePbFusion/agent/pdf_processor/outputs/qwen_output/4_reconstruct_tables_data/reconstruct_csv_tables_clean"
csv_files = glob.glob(os.path.join(input_dir, "*.csv"))
if not csv_files:
    print(f"No CSV files found in {input_dir}")
else:
    # Process each file
    saved_files = 0
    total_pb_data_points = 0

    for csv_file in csv_files:
        pb_count = validate_and_clean_csv(csv_file, output_dir)
        
        if pb_count > 0:
            saved_files += 1
            total_pb_data_points += pb_count
            print(f"Processed and saved: {os.path.basename(csv_file)} with {pb_count} valid Pb data points")

# Print summary
print("\nProcessing Summary:")
print(f"Total CSV files processed: {len(csv_files)}")
print(f"CSV files saved to {output_dir}: {saved_files}")
print(f"Total valid Pb data points: {total_pb_data_points}")

Processed and saved: j.marpolbul.2014.06.013.csv with 24 valid Pb data points
Missing required columns in /home/lym/PbCirculation/MarinePbFusion/agent/pdf_processor/outputs/qwen_output/4_reconstruct_tables_data/reconstruct_csv_tables/j.saa.2020.119190.csv. Skipping file.
Missing required columns in /home/lym/PbCirculation/MarinePbFusion/agent/pdf_processor/outputs/qwen_output/4_reconstruct_tables_data/reconstruct_csv_tables/j.marpolbul.2016.09.027.csv. Skipping file.
Missing required columns in /home/lym/PbCirculation/MarinePbFusion/agent/pdf_processor/outputs/qwen_output/4_reconstruct_tables_data/reconstruct_csv_tables/j.epsl.2014.04.030.csv. Skipping file.
Missing required columns in /home/lym/PbCirculation/MarinePbFusion/agent/pdf_processor/outputs/qwen_output/4_reconstruct_tables_data/reconstruct_csv_tables/j.ancene.2019.100234.csv. Skipping file.
Missing required columns in /home/lym/PbCirculation/MarinePbFusion/agent/pdf_processor/outputs/qwen_output/4_reconstruct_tables_data/rec